[View in Colaboratory](https://colab.research.google.com/github/nemo628/sentiment_analysis/blob/master/sentiment_analysis.ipynb)

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from google.colab import files
uploaded = files.upload()

Using TensorFlow backend.


Saving Sentiment.csv to Sentiment.csv


In [0]:
import io
data=pd.read_csv(io.StringIO(uploaded['Sentiment.csv'].decode('utf-8')))
data=data[['text', 'sentiment']]

In [3]:
print(data.head(5))
print(data.size)

                                                text sentiment
0  RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1  RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2  RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3  RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4  RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
27742


In [4]:
data=data[data['sentiment']!="Neutral"]
print(data.size)
print(data[data['sentiment']=="Positive"].size)
print(data[data['sentiment']=="Negative"].size)

21458
4472
16986


In [5]:
data['text']=data['text'].apply(lambda x: x.lower())
data['text']=data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))
print(data.head(5))

                                                text sentiment
1  rt scottwalker didnt catch the full gopdebate ...  Positive
3  rt robgeorge that carly fiorina is trending  h...  Positive
4  rt danscavino gopdebate w realdonaldtrump deli...  Positive
5  rt gregabbotttx tedcruz on my first day i will...  Positive
6  rt warriorwoman91 i liked her and was happy wh...  Negative


In [6]:
for idx, row in data.iterrows():
    row[0]=row[0].replace('rt', '')
print(data.head(5))    

                                                text sentiment
1   scottwalker didnt catch the full gopdebate la...  Positive
3   robgeorge that carly fiorina is trending  hou...  Positive
4   danscavino gopdebate w realdonaldtrump delive...  Positive
5   gregabbotttx tedcruz on my first day i will r...  Positive
6   warriorwoman91 i liked her and was happy when...  Negative


In [7]:
max_features=2000
tokenizer= Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X=tokenizer.texts_to_sequences(data['text'].values)
X =pad_sequences(X)
print(X[0])
print(X.shape[1])

[   0    0    0    0    0    0    0    0    0    0    0  358  120    1
  711    2   39   58  234   36  207    6  172 1740   12 1301 1386  732]
28


In [8]:
print(X.shape)
print(X.shape[1])

(10729, 28)
28


In [9]:
embed_dim=128
lstm_out=196

model=Sequential()
model.add(Embedding(max_features, embed_dim,input_length= X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout= 0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [11]:
batch_size= 32
model.fit(X_train, Y_train, epochs=10, batch_size=batch_size, verbose=2)

Epoch 1/10
 - 21s - loss: 0.4376 - acc: 0.8134
Epoch 2/10
 - 20s - loss: 0.3275 - acc: 0.8651
Epoch 3/10
 - 20s - loss: 0.2871 - acc: 0.8794
Epoch 4/10
 - 19s - loss: 0.2544 - acc: 0.8962
Epoch 5/10
 - 19s - loss: 0.2343 - acc: 0.9035
Epoch 6/10
 - 19s - loss: 0.2046 - acc: 0.9196
Epoch 7/10
 - 19s - loss: 0.1857 - acc: 0.9254
Epoch 8/10
 - 19s - loss: 0.1689 - acc: 0.9320
Epoch 9/10
 - 18s - loss: 0.1567 - acc: 0.9366
Epoch 10/10
 - 19s - loss: 0.1458 - acc: 0.9396


In [0]:
import h5py

In [0]:
model.save_weights('Sentiment_Analysis.weights.h5')

In [14]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.51
acc: 0.83


In [0]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

In [19]:
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 60.517799352750814 %
neg_acc 90.51217464315701 %


In [21]:
twt=["Vote for the man who promises least; he'll be the least disappointing."]
twt=tokenizer.texts_to_sequences(twt)
twt=pad_sequences(twt, maxlen=28, dtype='int', value=0)
print(twt)

sentiment=model.predict(twt, batch_size=1, verbose=2)[0]

if(np.argmax(sentiment)==0):
  print("negative")
elif(np.argmax(sentiment)==0):
  print("positive")
  

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  200  14   1 229  52 859  34   1 859 179]]
negative
